# Load the map

We will first load the map file, which gives the terrain information. Choose the right file.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
# from google.colab.patches import cv2_imshow
# from google.colab import output
import ipywidgets as widgets
from IPython.display import display

# Load the map image. Use 0 for grayscale, single-channel
map = cv2.imread("map.png", 0)
HEIGHT, WIDTH = map.shape # get the dimensions of the map image

# Set a random initial robot pose
rx, ry, rtheta = (WIDTH / 4, HEIGHT / 4, 0)

print(map) # check the map

Below code is written to check the map and initial robot position.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(map, cmap='gray')
ax.scatter([rx], [ry], color='green', s=100, edgecolors='black')  # Green dot for robot
ax.set_axis_off()
plt.show()

In reality, you have uneven terrain. So, your robot will not move as you command. You can model this uncertainty using noise. We will use Gaussian noise.

In [3]:
# Step size in pixels and turn in radians
STEP = 5
TURN = np.radians(25)

# Standard deviations for motion and sensing noise
SIGMA_STEP = 0.5
SIGMA_TURN = np.radians(5)

# Move robot with noise
def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = fwd + np.random.normal(0, SIGMA_STEP)
    turn_noisy = turn + np.random.normal(0, SIGMA_TURN)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)
    rtheta += turn_noisy
    return rx, ry, rtheta

# Particle Filter Initialization

Now, we are ready to use particle filter to localize our robot. First of all, we need to initialize the particles.

In [ ]:
NUM_PARTICLES = 3000

# Initialize particles
def init():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array((WIDTH, HEIGHT, np.radians(360)))
    return particles

# Plot map and particles
def plot_on_map(map, rx, ry, particles):
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map, cmap='gray')
    ax.scatter(particles[:, 0], particles[:, 1], color='blue', s=1)
    ax.scatter([rx], [ry], color='green', s=100, edgecolors='black') # truth
    if len(particles) > 0:
        px = np.mean(particles[:, 0])
        py = np.mean(particles[:, 1])
        ax.scatter([px], [py], color='red', s=100, edgecolors='black') # estimation
    ax.axis('off')
    plt.show()

particles = init()
plot_on_map(map, rx, ry, particles)

When we get a command from keyboards, we update the particles based on the command.

In [5]:
# Move particles based on the command
def move_particles(particles, fwd, turn):
    particles[:, 0] += fwd * np.cos(particles[:, 2])
    particles[:, 1] += fwd * np.sin(particles[:, 2])
    particles[:, 2] += turn
    return particles

# Sensor measurement model

Now, we need to model how the robot obtains its elevation. The sensor readings should have noise, and the noise is modeled as Gaussian.


In [6]:
SIGMA_SENSOR = 5

def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)

    # when x and y are out of range, get the value at the edge
    if x > WIDTH - 1:
      x = WIDTH - 1
    if x < 0:
      x = 0
    if y > HEIGHT - 1:
      y = HEIGHT - 1
    if y < 0:
      y = 0

    if noisy:
        return map[y,x] + np.random.normal(0.0, SIGMA_SENSOR, 1)
    return map[y,x]

# Resample the particles

Now we will use particle weights to make the estimation better.

In [7]:
# Compute particle weights
def compute_weights(particles, robot_sensor):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        particle_sensor = sense(particles[i,0], particles[i,1])
        errors[i] = abs(robot_sensor - particle_sensor)
    weights = np.max(errors) - errors

    # Kill off particles on edge
    weights[
        (particles[:,0] <= 0) |
        (particles[:,0] >= WIDTH-1) |
        (particles[:,1] <= 0) |
        (particles[:,1] >= HEIGHT-1)
    ] = 0.0

    # Increase sensitivity
    weights = weights ** 3
    return weights

Now, the particles with higher weights would have been sampled more. The particles with lower weights would have not been sampled that much.

In [8]:
# Resample particles
def resample(particles, weights):
    probabilities = weights / np.sum(weights)
    new_index = np.random.choice(NUM_PARTICLES, size=NUM_PARTICLES, p=probabilities)
    return particles[new_index, :]

Next, we will perturb the particles. This will improve estimation performance because we have lots of uncertainties. Here, we have uncertainteis from the motion model and sensor readings. So, we need to address those uncertainties in estimation. Again, we will use Gaussian noise.

In [9]:
SIGMA_PARTICLE_STEP = 2 #2
SIGMA_PARTICLE_TURN = np.pi / 24 #np.pi / 24

def add_noise(particles):
    noise = np.concatenate((
        np.random.normal(0, SIGMA_PARTICLE_STEP, (NUM_PARTICLES,1)),
        np.random.normal(0, SIGMA_PARTICLE_STEP, (NUM_PARTICLES,1)),
        np.random.normal(0, SIGMA_PARTICLE_TURN, (NUM_PARTICLES,1)),
        ),
        axis=1
    )
    particles += noise
    return particles

# Let the robot move with keyboards with Particle Filter

We will manipulate the robot using keyboards.

In [10]:
# Define button actions
def move_forward(b):
    global rx, ry, rtheta, particles
    fwd, turn = STEP, 0
    update_robot_and_particles(fwd, turn)

def turn_left(b):
    global rx, ry, rtheta, particles
    fwd, turn = 0, -TURN
    update_robot_and_particles(fwd, turn)

def turn_right(b):
    global rx, ry, rtheta, particles
    fwd, turn = 0, TURN
    update_robot_and_particles(fwd, turn)

def update_robot_and_particles(fwd, turn):
    global rx, ry, rtheta, particles
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    robot_sensor = sense(rx, ry, noisy=True)
    weights = compute_weights(particles, robot_sensor)
    particles = resample(particles, weights)
    particles = add_noise(particles)
    plot_on_map(map, rx, ry, particles)
    display(button_forward, button_left, button_right)

# Create buttons for robot control
button_forward = widgets.Button(description="Move Forward")
button_left = widgets.Button(description="Turn Left")
button_right = widgets.Button(description="Turn Right")

# Set button on_click events
button_forward.on_click(move_forward)
button_left.on_click(turn_left)
button_right.on_click(turn_right)

# Main code to run particle filter

In [ ]:
# Start by showing the image with the dot in the initial position
plot_on_map(map, rx, ry, particles)

# Display buttons
display(button_forward, button_left, button_right)
